In [1]:
import sysconfig
import os
import numpy as np
import pandas as pd
import json
import distutils
import scrapy
import datetime
import requests
import json
import logging
import string
import re
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from requests import Request
from math import sin, cos, sqrt, atan2, radians
from decimal import Decimal
import math

# approximate radius of earth in km
R = 6373.0



In [2]:
class Stayz_Listing(scrapy.Item):
    property_id = scrapy.Field()
    lat = scrapy.Field()
    lng = scrapy.Field()
    init_price = scrapy.Field()
    guests = scrapy.Field()
    heading = scrapy.Field()
    description_full = scrapy.Field()
    url = scrapy.Field()
    bedrooms = scrapy.Field()
    beds = scrapy.Field()
    bathrooms = scrapy.Field()
    property_type = scrapy.Field()
    reviews = scrapy.Field()
    #rating = scrapy.Field()
    #suburb = scrapy.Field()
    #postcode = scrapy.Field()
    #google_addr = scrapy.Field()
    #house_specs = scrapy.Field()
    scraped_date = scrapy.Field()
    syd_brg_deg = scrapy.Field()
    syd_brg = scrapy.Field()
    syd_dist_km = scrapy.Field()
    
        

In [3]:
# Setup a pipeline
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('stayz.jl','w')
        
    def close_spider(self, spider):
        self.file.close()
        
    def processitem(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [4]:
# http://www.thecodeknight.com/post_categories/search/posts/scrapy_python

class StayzSpider(scrapy.Spider):
    name = 'stayz_crawler'
    
    # Full scrape - NSW
    start_urls = ['https://www.stayz.com.au/accommodation/nsw']
    
    # Forbes - has 3 properties on one listing page
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/forbes']
    
    # Orange - has 122 properties on 3 listing pages
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/orange/']
    
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/mudgee'
    #             ,'https://www.stayz.com.au/accommodation/nsw/explorer-country/bathurst'
    #             ,'https://www.stayz.com.au/accommodation/nsw/north-coast/coffs-harbour'
    #             ,'https://www.stayz.com.au/accommodation/nsw/north-coast/port-macquarie']
    
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/central-coast/gosford/'
    #              ,'https://www.stayz.com.au/accommodation/nsw/explorer-country/orange/'
    #             ]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, #Used for pipeline 1
        'FEED_FORMAT':'json', # Used for pipeline 2
        'FEED_URI': 'WebData/stayz_nsw_extract_' + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + '.json' #Used for pipeline 2
    }
    
    url_pages = []
    
    def calculate_initial_compass_bearing(self, pointA, pointB):
        """
        Calculates the bearing between two points.
        The formulae used is the following:
            θ = atan2(sin(Δlong).cos(lat2),
                      cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
        :Parameters:
          - `pointA: The tuple representing the latitude/longitude for the
            first point. Latitude and longitude must be in decimal degrees
          - `pointB: The tuple representing the latitude/longitude for the
            second point. Latitude and longitude must be in decimal degrees
        :Returns:
          The bearing in degrees
        :Returns Type:
          float
        """
        if (type(pointA) != tuple) or (type(pointB) != tuple):
            raise TypeError("Only tuples are supported as arguments")

        lat1 = math.radians(pointA[0])
        lat2 = math.radians(pointB[0])
        
        lon1 = math.radians(pointA[1])
        lon2 = math.radians(pointB[1])

        diffLong = math.radians(pointB[1] - pointA[1])

        x = math.sin(diffLong) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
                * math.cos(lat2) * math.cos(diffLong))

        initial_bearing = math.atan2(x, y)

        # Now we have the initial bearing but math.atan2 return values
        # from -180° to + 180° which is not what we want for a compass bearing
        # The solution is to normalize the initial bearing as shown below
        initial_bearing = math.degrees(initial_bearing)
        compass_bearing = (initial_bearing + 360) % 360
        
        
        # Calculate the straight line distance in km
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        syd_distance_km = int(R * c)
        
        
        if compass_bearing >= 0.00 and compass_bearing < 22.50:
            compass_dir = 'N'
        elif compass_bearing >= 22.50 and compass_bearing < 67.5:
            compass_dir = 'NE'
        elif compass_bearing >= 67.5 and compass_bearing < 112.50:
            compass_dir = 'E'
        elif compass_bearing >= 112.5 and compass_bearing < 157.50:
            compass_dir = 'SE'
        elif compass_bearing >= 157.5 and compass_bearing < 202.50:
            compass_dir = 'S'
        elif compass_bearing >= 202.5 and compass_bearing < 247.50:
            compass_dir = 'SW'
        elif compass_bearing >= 247.5 and compass_bearing < 292.50:
            compass_dir = 'W'
        elif compass_bearing >= 292.5 and compass_bearing < 337.50:
            compass_dir = 'NW'
        elif compass_bearing >= 337.5 and compass_bearing <= 360.00:
            compass_dir = 'N'
        else:
            compass_dir = 'UN'

        return (int(compass_bearing), compass_dir, int(syd_distance_km))
    
    def parse(self, response):
        
        sel = Selector(response)
               
        urls = sel.xpath('//section[@class="c-search-results__main"]/div/article/div/div/div/h3/a/@href').extract()
                
        for u in urls:
        
            request = scrapy.Request('https://www.stayz.com.au/'+ u, callback=self.parse_results )
            
            yield request
            
        # Get the link to the 'Next' page of listings
        url_pages = sel.xpath('/html/body/div/main/div/section/div/nav/ul/li/a/@href').extract()
        
        if len(url_pages) > 0:
            nextPage = url_pages[-1]
        
            nextPageURL = 'https://www.stayz.com.au/'+ nextPage
        
            print("Next URL: " + str(nextPage))
        
            yield(scrapy.Request(nextPageURL, callback=self.parse))
            
    def strip_whitespace(self, st):
    
        st1 = re.sub('\n','',st.string() )
        st2 = re.sub(' +',' ', st1.string() )
        return st2    
                
    def parse_results(self, response):
        
        # Property ID - unique for each property
        p_id1 = response.selector.xpath('//article/header/ol/li/span/span/text()').extract_first()
        p_id2 = re.sub('\n','',p_id1)
        p_id3 = re.sub(' +',' ', p_id2)
        p_id4 = re.search('\d+',p_id3)
        p_id = p_id4.group(0)

        # Geographic location - not exact location, within a block of actual
        # Error handling - if no lat/lon specified place in middle of Botany Harbour
        p_lat = response.selector.xpath('//*[@class="c-map__container"]/@lat').extract_first()
        if p_lat is None:
            p_lat = '-33.990'
        p_lng = response.selector.xpath('//*[@class="c-map__container"]/@lng').extract_first() 
        if p_lng is None:
            p_lng = '151.180'

        #print("Distance to Sydney: {0} km" .format(p_syd_distance_km))
        
        p_compass_bearing, p_dir, p_syd_distance_km = self.calculate_initial_compass_bearing((Decimal(-33.990),Decimal(151.180)),(Decimal(p_lat),Decimal(p_lng)))
        #print("Distance: {2} Compass: {0} Heading: {1}" .format(p_compass_bearing, p_dir, p_syd_distance_km))
        

        # House configuration - number of guests
        p_configuration = response.css(".c-facet__label.u-display--block::text").extract_first()
        p_configuration = re.sub('\n','',p_configuration)
        p_configuration = re.sub(' +',' ', p_configuration)
        p_group = re.search('\d+', p_configuration)
        p_guests = p_group.group(0)
        
        p_heading = response.css(".c-heading--brand.u-h2::text").extract_first()
        
        # Property type from the sub-heading - 'House, Mudgee'
        p_pt_1 = response.selector.xpath('///html/body/main/section/div/article/header/p/small/text()').extract_first()
        
        if p_pt_1 is not None:
            p_pt_1 = re.sub('\n','', p_pt_1)
            p_pt_1 = re.sub(' +',' ', p_pt_1)
            if re.search('B&B', p_pt_1):
                p_property_type = 'B&B'
            else:
                p_pt_2 = re.search('\w+',p_pt_1)
                p_property_type = p_pt_2.group(0)
        else:
            p_property_type = 'Unknown'
        
        p_house_specs = response.selector.xpath('//div[@class="c-facets c-facets--inline"]/div/span/span/text()').extract()
        
        # Initialize variables
        n_bedrooms = 0
        n_beds = 0
        n_bathrooms = 0
        
        for j in p_house_specs:
            if re.search('bedroom',j):
                # Extract number of guests
                n_br = re.search('\d+',j)
                n_bedrooms = n_br.group(0)
                
            if re.search('bed[s]?$',j):
                # Extract number of guests
                n_bd = re.search('\d+',j)
                n_beds = n_bd.group(0)
                
            if re.search('bathroom',j):
                # Extract number of guests
                n_bath = re.search('\d+',j)
                n_bathrooms = n_bath.group(0)
                
                
        # The basic price - 'From $xxx per night'        
        p_ip_1 = response.selector.xpath('//header[@class="c-quote__header"]/p/span/span/span[@class="u-h1"]/text()').extract_first()
        
        p_init_price = 0
        
        if p_ip_1 is not None:
            p_ip_3 = p_ip_1.replace(',','')
            p_ip_4 = re.search('\d+',p_ip_3)
            p_init_price = p_ip_4.group(0)
        else:
            p_init_price = 0
        
        # Ratings and reviews
        p_rating = response.selector.xpath('//header[@class="c-reviews__header"]/span/span/span[@class="c-facet__label"]/span/text()').extract_first()
        
        p_r1 = response.selector.xpath('//html/body/main/section/div/article/header/p[2]/span/span[2]/small/text()').extract_first()
        
        # Initialise
        p_reviews = 0
        
        if p_r1 is not None:
            p_r2 = re.sub('\n','',p_r1)
            p_r2 = re.sub(' +',' ',p_r2)
            p_r3 = re.search('\d+', p_r2)
            p_reviews = p_r3.group(0)
        else:
            p_reviews = 0
        
        # Full text of property details
        p_description_full = response.xpath('//div[@id="property-description-body"]/p/text()').extract_first()
        
        # Reverse geocoding on Google API for street address:
        latlng = "" + p_lat + "," + p_lng 
        
        print("Scanning Property ID: " + p_id + "-" + response.url)
        
        p = Stayz_Listing()
        p['property_id'] = p_id
        p['lat'] = p_lat
        p['lng'] = p_lng
        p['guests'] = p_guests
        p['heading'] = p_heading
        p['url'] = response.url
        p['init_price'] = p_init_price
        p['bedrooms'] = n_bedrooms
        p['beds'] = n_beds
        p['bathrooms'] = n_bathrooms
        p['description_full'] = p_description_full
        p['property_type'] = p_property_type
        #p['rating'] = p_rating
        p['reviews'] = p_reviews
        #p['suburb'] = p_suburb
        #p['postcode'] = p_postcode
        #p['house_specs'] = p_house_specs
        #p['google_addr'] = google_addr.text
        p['syd_dist_km'] = p_syd_distance_km
        p['syd_brg_deg'] = p_compass_bearing
        p['syd_brg'] = p_dir
        p['scraped_date'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        yield p        


In [5]:
process = CrawlerProcess({ 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)' })

process.crawl(StayzSpider) 
process.start()

2018-02-26 10:23:19 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-02-26 10:23:19 [scrapy.utils.log] INFO: Versions: lxml 4.1.0.0, libxml2 2.9.4, cssselect 1.0.3, parsel 1.3.1, w3lib 1.18.0, Twisted 16.6.0, Python 3.6.3 |Anaconda, Inc.| (default, Oct  6 2017, 12:04:38) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 17.2.0 (OpenSSL 1.0.2l  25 May 2017), cryptography 2.0.3, Platform Darwin-17.4.0-x86_64-i386-64bit
2018-02-26 10:23:19 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'WebData/stayz_nsw_extract_2018-02-26_10-23.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


Next URL: /accommodation/nsw?page=2
Distance: 121 Compass: 201 Heading: S
Scanning Property ID: 9064241-https://www.stayz.com.au//accommodation/nsw/south-coast/jervis-bay/9064241
Distance: 84 Compass: 291 Heading: W
Scanning Property ID: 9132273-https://www.stayz.com.au//accommodation/nsw/blue-mountains/wentworth-falls/9132273
Distance: 102 Compass: 215 Heading: SW
Scanning Property ID: 9132090-https://www.stayz.com.au//accommodation/nsw/south-coast/kangaroo-valley/9132090
Distance: 15 Compass: 8 Heading: N
Scanning Property ID: 9204526-https://www.stayz.com.au//accommodation/nsw/sydney/mcmahons-point/9204526
Distance: 286 Compass: 200 Heading: S
Scanning Property ID: 9082905-https://www.stayz.com.au//accommodation/nsw/south-coast/bermagui/9082905
Distance: 198 Compass: 202 Heading: SW
Scanning Property ID: 9195589-https://www.stayz.com.au//accommodation/nsw/south-coast/depotbeach/9195589
Distance: 440 Compass: 24 Heading: NE
Scanning Property ID: 9203655-https://www.stayz.com.au//acco

Distance: 96 Compass: 208 Heading: SW
Scanning Property ID: 9173464-https://www.stayz.com.au//accommodation/nsw/south-coast/berry/9173464
Distance: 221 Compass: 35 Heading: NE
Scanning Property ID: 9061166-https://www.stayz.com.au//accommodation/nsw/north-coast/blueys-beach/9061166
Distance: 100 Compass: 213 Heading: SW
Scanning Property ID: 9152089-https://www.stayz.com.au//accommodation/nsw/south-coast/kangaroo-valley/9152089
Distance: 52 Compass: 358 Heading: N
Scanning Property ID: 9155080-https://www.stayz.com.au//accommodation/nsw/sydney/hawkesburyriver/9155080
Distance: 101 Compass: 215 Heading: SW
Scanning Property ID: 9018551-https://www.stayz.com.au//accommodation/nsw/south-coast/kangaroo-valley/9018551
Next URL: /accommodation/nsw?page=5
Distance: 41 Compass: 3 Heading: N
Scanning Property ID: 9173589-https://www.stayz.com.au//accommodation/nsw/sydney/cottage-point/9173589
Distance: 89 Compass: 292 Heading: W
Scanning Property ID: 9023254-https://www.stayz.com.au//accommodat

Distance: 647 Compass: 20 Heading: N
Scanning Property ID: 9180438-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/mullumbimby/9180438
Distance: 338 Compass: 199 Heading: S
Scanning Property ID: 9104775-https://www.stayz.com.au//accommodation/nsw/south-coast/tura-beach/9104775
Distance: 64 Compass: 21 Heading: N
Scanning Property ID: 9013260-https://www.stayz.com.au//accommodation/nsw/central-coast/terrigal/9013260
Distance: 79 Compass: 22 Heading: N
Scanning Property ID: 9068055-https://www.stayz.com.au//accommodation/nsw/central-coast/the-entrance/9068055
Distance: 933 Compass: 281 Heading: W
Scanning Property ID: 9149544-https://www.stayz.com.au//accommodation/nsw/explorer-country/broken-hill/9149544
Distance: 162 Compass: 203 Heading: SW
Scanning Property ID: 9155707-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook-beach/9155707
Distance: 546 Compass: 22 Heading: NE
Scanning Property ID: 9163849-https://www.stayz.com.au//accommodation/nsw/nort

Distance: 130 Compass: 6 Heading: N
Scanning Property ID: 9161684-https://www.stayz.com.au//accommodation/nsw/hunter/cessnock/9161684
Distance: 118 Compass: 20 Heading: N
Scanning Property ID: 9065508-https://www.stayz.com.au//accommodation/nsw/hunter/lake-macquarie/9065508
Distance: 546 Compass: 22 Heading: NE
Scanning Property ID: 9171605-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/yamba/9171605
Distance: 35 Compass: 14 Heading: N
Scanning Property ID: 9181793-https://www.stayz.com.au//accommodation/nsw/sydney/ingleside/9181793
Distance: 383 Compass: 222 Heading: SW
Scanning Property ID: 9085676-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/thredbo/9085676
Distance: 309 Compass: 30 Heading: NE
Scanning Property ID: 9151659-https://www.stayz.com.au//accommodation/nsw/north-coast/bonny-hills/9151659
Distance: 161 Compass: 203 Heading: SW
Scanning Property ID: 9195976-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook-beach/9195976


...

Distance: 91 Compass: 219 Heading: SW
Scanning Property ID: 9097719-https://www.stayz.com.au//accommodation/nsw/southern-highlands/robertson/9097719
Distance: 81 Compass: 22 Heading: NE
Scanning Property ID: 9139495-https://www.stayz.com.au//accommodation/nsw/central-coast/magenta/9139495
Distance: 358 Compass: 219 Heading: SW
Scanning Property ID: 9109724-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/jindabyne/9109724
Distance: 22 Compass: 25 Heading: NE
Scanning Property ID: 9169462-https://www.stayz.com.au//accommodation/nsw/sydney/manly/9169462
Distance: 455 Compass: 24 Heading: NE
Scanning Property ID: 9169448-https://www.stayz.com.au//accommodation/nsw/north-coast/coffs-harbour/9169448
Distance: 223 Compass: 203 Heading: SW
Scanning Property ID: 9171096-https://www.stayz.com.au//accommodation/nsw/south-coast/tomakin/9171096
Distance: 59 Compass: 22 Heading: N
Scanning Property ID: 9011380-https://www.stayz.com.au//accommodation/nsw/central-coast/macmasters-beach/901

Distance: 42 Compass: 19 Heading: N
Scanning Property ID: 9180101-https://www.stayz.com.au//accommodation/nsw/sydney/avalon-beach/9180101
Distance: 101 Compass: 217 Heading: SW
Scanning Property ID: 9213237-https://www.stayz.com.au//accommodation/nsw/south-coast/kangaroo-valley/9213237
Distance: 474 Compass: 24 Heading: NE
Scanning Property ID: 9119640-https://www.stayz.com.au//accommodation/nsw/north-coast/woolgoolga/9119640
Distance: 116 Compass: 5 Heading: N
Scanning Property ID: 9085833-https://www.stayz.com.au//accommodation/nsw/hunter/hunter-valley/9085833
Distance: 673 Compass: 20 Heading: N
Scanning Property ID: 9068971-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/casuarina/9068971
Distance: 214 Compass: 204 Heading: SW
Scanning Property ID: 9174768-https://www.stayz.com.au//accommodation/nsw/south-coast/batemans-bay/9174768
Distance: 209 Compass: 204 Heading: SW
Scanning Property ID: 9004822-https://www.stayz.com.au//accommodation/nsw/south-coast/batem

Distance: 165 Compass: 33 Heading: NE
Scanning Property ID: 9168439-https://www.stayz.com.au//accommodation/nsw/port-stephens/fingal-bay/9168439
Distance: 132 Compass: 201 Heading: S
Scanning Property ID: 9182090-https://www.stayz.com.au//accommodation/nsw/south-coast/jervis-bay/9182090
Distance: 382 Compass: 222 Heading: SW
Scanning Property ID: 9087325-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/thredbo/9087325
Distance: 163 Compass: 203 Heading: SW
Scanning Property ID: 9102083-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook/9102083
Distance: 165 Compass: 13 Heading: N
Scanning Property ID: 9208654-https://www.stayz.com.au//accommodation/nsw/hunter/hunter-valley/9208654
Distance: 161 Compass: 203 Heading: SW
Scanning Property ID: 9144936-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook/9144936
Distance: 448 Compass: 24 Heading: NE
Scanning Property ID: 9110723-https://www.stayz.com.au//accommodation/nsw/north-coast/coffs-harbour/911

Distance: 173 Compass: 319 Heading: NW
Scanning Property ID: 9136610-https://www.stayz.com.au//accommodation/nsw/explorer-country/mudgee/9136610
Distance: 673 Compass: 20 Heading: N
Scanning Property ID: 9209693-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/casuarina/9209693
Distance: 57 Compass: 15 Heading: N
Scanning Property ID: 9169037-https://www.stayz.com.au//accommodation/nsw/central-coast/central-coast-waterfront/9169037
Distance: 134 Compass: 201 Heading: S
Scanning Property ID: 9192049-https://www.stayz.com.au//accommodation/nsw/south-coast/sanctuary-point/9192049
Distance: 118 Compass: 195 Heading: S
Scanning Property ID: 9017363-https://www.stayz.com.au//accommodation/nsw/south-coast/currarong/9017363
Distance: 322 Compass: 356 Heading: N
Scanning Property ID: 9181098-https://www.stayz.com.au//accommodation/nsw/new-england/tamworth/9181098
Distance: 370 Compass: 249 Heading: W
Scanning Property ID: 9212141-https://www.stayz.com.au//accommodation/nsw/

Scanning Property ID: 9165786-https://www.stayz.com.au//accommodation/nsw/north-coast/valla-beach/9165786
Distance: 382 Compass: 222 Heading: SW
Scanning Property ID: 9006125-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/thredbo/9006125
Distance: 161 Compass: 203 Heading: SW
Scanning Property ID: 9205519-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook-beach/9205519
Distance: 12 Compass: 15 Heading: N
Scanning Property ID: 9079910-https://www.stayz.com.au//accommodation/nsw/sydney/darlinghurst/9079910
Distance: 176 Compass: 203 Heading: SW
Scanning Property ID: 9053423-https://www.stayz.com.au//accommodation/nsw/south-coast/lake-tabourie/9053423
Distance: 321 Compass: 224 Heading: SW
Scanning Property ID: 9186987-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/adaminaby/9186987
Distance: 82 Compass: 291 Heading: W
Scanning Property ID: 9026937-https://www.stayz.com.au//accommodation/nsw/blue-mountains/leura/9026937
Distance: 110 Compass: 200 H